# HeadCheck Analysis v1.0

*Author: Eric G. Suchanek, PhD. 6/10/19*

## Purpose
This program allows a user to analyze Observation .fits files corresponding to images acquired by Drs Shane and Babcock at Chews Ridge Observatory. It is capable of listing individual files, reviewing their header contents and provides for user input of observation time, observation airmass, beginning hour angle and exposure time. The program then computes the various time differences:
* Entered and recorded Observation time
* Entered and recorded Exposure time
* Entered and recorded Airmass

The program then computes the object's airmass by using the _astropy_ class library to calculate the object's altitude at the observation time, with a location set to the coordinates of OOS. This is used to compute a difference between the airmass placed in the header. The overall results are then displayed.

In [1]:
# 
# Create a program to read input from the user per the Headcheck manual, load the .fits file and then calculate the 
# various times and positions needed.

from glob import glob
import os
import time
import datetime
import sys

import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, Latitude, Longitude
from astropy.io import fits
from astropy.time import Time

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from astropy.visualization import (SqrtStretch,ImageNormalize,PercentileInterval)

#
# DST calculation for a given date
# expects date to be .unix format

def is_dst(date):
    return bool(time.localtime(date).tm_isdst)

# Main code to input and analyze .fits files per the Headcheck protocol
# Set up our Observatory location.
# OOS coordinates taken from Google Earth
# convert to geodetic coords via the EarthLocation class. We need this for sidereal calculations

sitelong = '-121:34:00'
sitelat = '36:18:20'
height = 1525*u.m
observatory_location = EarthLocation(sitelong, sitelat, height=height)

# Base constant UTC offset for US/Pacific. This can change to -7 hour when DST is in effect.
# the local var utcoffset reflects the ACTUAL utcoffset for the given date, taking DST into account
UTC_offset = -8*u.hour

# default path is local dir
# global vars
_path = './'
_ra_str = ''
_dec_str = ''
_airmass = 0.0
_time_obs = ''
_filename = ''
_exposure = 0.0
_utc_time_obs = ''
_bha = ''
_jd = 0.0

_file_list = [] # hold all files entered for processing
_file_count = 0

# toggle image plotting
Plot_images = False

do_process = False
getting_input = True

path_prompt = 'Enter a path for the data files relative to the path of this script: '
fname_prompt = 'Enter a file name (no path): '
bha_prompt = 'Enter beginning hour angle (hh:mm:ss): '
ra_prompt = 'Enter RA (hh:mm:ss): '
dec_prompt = 'Enter Dec (deg:mm:ss): '
airmass_prompt = 'Enter Airmass: '
exp_prompt = 'Enter Exposure (sec): '
jd_prompt = 'Enter Julian Day: '
cmd_prompt = 'Enter a command, q to exit, P to exit and process: '

cmd_list = "\n******************* HeadCheck Input *******************\n\
p --- Enter path\n\
f --- Enter a filename to be processed \n\
l --- List files\n\
h --- View Header \n\
t --- Enter date & time \n\
j --- Enter Julian Day \n\
b --- Enter Beginning Hour Angle \n\
s --- Enter Airmass \n\
x --- Enter Exposure (sec) \n\
v --- Review Input \n\
P --- Process input \n\
z --- Reset all \n\
q --- Quit"

from pathlib import Path

def exit():
    getting_input = False
    do_process = True
    return

def get_path():
    path = input(path_prompt)
    my_path = Path(path)
    if my_path.exists():
        ;
    else:
        print('Path: ', path, 'cannot be found!')
        #input('Press Return to continue')
        return ''
    return path

# list only .fits files
def list_files(path):
    i = 0
    files = glob(path + '*.fits')
    if len(files) == 0:
        print("No files found.")
    else:
        print('\nFiles in path:', path)
        for file in files:
            print(file)
            i+=1
    return(i)

# input() returns a STRING. Convert to proper type as needed!

def enter_ra():
    return input(ra_prompt)

def enter_bha():
    return input(bha_prompt)
    
def enter_jd():
    return input(jd_prompt)

def enter_dec():
    return input(dec_prompt)

def enter_airmass():
    return float(input(airmass_prompt))

def enter_exposure():
    return float(input(exp_prompt))

def add_file():
    _fname = input(fname_prompt)
    _file = _path + _fname      
    my_file = Path(_file)
    if my_file.is_file():
        return _file
    else:
        print('File', _file, 'cannot be opened. Check path and filename!')
        input('Press Return to continue')
        _fname = ''
        return ''
    return _file

def reset_all():
    _airmass = -1.0
    _ra_str = ''
    _dec_str = ''
    _time_obs = ''
    _file_list = []
    _filename = ''
    _path = './'
    _exposure = 0.0
    _exp = 0.0
    _file_count = 0
    _bha = ''
    _jd = -1.0
    _utc_time_obs = ''
    do_process = False
    getting_input = True
    return

def enter_time_obs():
    _timeobs = input("Enter date/time of the observation, eg 2019-3-09 11:34:22.1 ")
    try:
        _tst = Time(_timeobs,location=observatory_location)
    except:
        print('Invalid time format. Try again')
        #input('Press return to continue.')
        return('')
    return _timeobs

def review_input():
    print('*********** Current Parameters *************')
    print(' Path: ', _path)
    print(' File: ', _filename)
    print(' Date/time: ', _utc_time_obs)
    print(' Julian Day: ', _jd)
    print(' Airmass: ', _airmass)
    print(' Beginning HA: ', _bha)
    print(' Exposure: ', _exposure)
    input(' Enter Return to continue.')
    return 

def review_header(fits_file):
    data, hdr = fits.getdata(fits_file, header=True, ext=0)
    date_obs = hdr['date-obs']
    exptime = hdr['exptime']
    obj_name = hdr['object']
    airmass = hdr['airmass']
    obj_ra = hdr['ra']
    obj_dec = hdr['dec']
    
    print('\n')
    print('File:          ', fits_file)
    print('Object:        ', obj_name)
    print('Obs date UTC:  ', date_obs)
    print('Exposure:      ', exptime)
    print('RA:            ', obj_ra)
    print('Dec:           ', obj_dec)
    print('Airmass:       ', airmass)
    fig = plt.figure(figsize=(7,3),dpi=120)
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data, origin='lower')
    plt.show()
    input('Press return to continue')
    return

# reset globals
reset_all()

# main input loop begins
while getting_input is True:
    # os.system('clear')  # on linux / os x
    print(cmd_list)
    cmd = input(cmd_prompt)    
    if cmd is 'q':
        getting_input = False
        do_process = False
        
    elif cmd is 'P':
        getting_input = False
        do_process = True
        
    elif cmd is 'f':
        _filename = add_file()
        if _filename is not '':
            _file_list.append(_filename)
            
    elif cmd is 'l':
        _file_count = list_files(_path)
        
    elif cmd is 'b':
        _bha = enter_bha()
        
    elif cmd is 'r':
        _ra_str = enter_ra()
        
    elif cmd is 'd':
        _dec_str = enter_dec()
        
    elif cmd is 'p':
        _path = get_path()
        
    elif cmd is 't':
        _utc_time_obs = enter_time_obs()
        
    elif cmd is 'j':
        _jd = enter_jd()
        
    elif cmd is 's':
        _airmass = enter_airmass()
        
    elif cmd is 'x':
        _exposure = enter_exposure()
        
    elif cmd is 'v':
        review_input()
        #input('Press return to continue')
        
    elif cmd is 'z':
        reset_all()
        
    elif cmd is 'h':
        if len(_filename) > 0:
            review_header(_filename)
        else:
            print('Enter a file first.\n')
            #input('Press return to continue')            
    else:
        print('Invalid Command!\n')

## Do the computations

if (do_process is True):
    # open the file and get the data block (image) and the header
    # the header is in block 0 of the fits file
    data, hdr = fits.getdata(_filename, header=True, ext=0)
    date_obs = hdr['date-obs']
    exptime = float(hdr['exptime'])
    obj_name = hdr['object']
    airmass = float(hdr['airmass'])
    obj_ra = hdr['ra']
    obj_dec = hdr['dec']
   
    # compute the .fits vs entered exposure times
    exposure_time = exptime*u.second 
    exposure_time_usr = _exposure*u.second
    exposure_time_usr_diff = exposure_time - exposure_time_usr
    
    # convert the .fits strings for ra and dec into actual angle objects for later use
    # the units library makes this possible, and enforces correct unit usage
    ra = Longitude(obj_ra, unit=u.hourangle)
    dec = Latitude(obj_dec, unit=u.deg)
    
    # reformat the ra and dec into nice strings for printing. Cosmetic
    ra_hms_str = ra.to_string(u.hour, precision=4)
    dec_dms_str = dec.to_string(u.degree, precision=4)
    
    # time calculations for the .fits data
    UTC_Obs = Time(date_obs,format='fits',location=observatory_location)
    UTC_mid_exposure_time = UTC_Obs + exposure_time / 2
    UTC_end_exposure_time = UTC_Obs + exposure_time

    # check if dst is in effect and set our utcoffset appropriately. Note use of proper units
    if is_dst(UTC_Obs.unix):
        utcoffset = UTC_offset + 1*u.hour
        dst_string = 'PDT'
    else:
        dst_string = 'PST'
        
    # compute time based on the input time from above   
    # the _usr variables represent parameters entered by the user above
    UTC_Obs_usr = Time(_utc_time_obs,format='fits',location=observatory_location)
    UTC_mid_exposure_time_usr = UTC_Obs_usr + exposure_time_usr / 2
    
    LZT_obs_usr = UTC_Obs_usr + utcoffset
    LZT_obs_mid_usr = UTC_mid_exposure_time_usr + utcoffset
    
    # times computed from the .fits header information
    LZT_obs = Time(UTC_Obs + utcoffset, location=observatory_location)
    LZT_obs_mid = LZT_obs + exposure_time / 2
    LZT_obs_end = LZT_obs + exposure_time
    
    sidereal_time_type = 'mean' # or 'apparent'. We use mean.  
    
    siderealTime_local_start = LZT_obs.sidereal_time(sidereal_time_type) 
    siderealTime_local_mid = LZT_obs_mid.sidereal_time(sidereal_time_type) 
    siderealTime_local_end = LZT_obs_end.sidereal_time(sidereal_time_type) 

    siderealTime_UTC_start = UTC_Obs.sidereal_time(siderealTime_type) 
    siderealTime_UTC_mid = UTC_mid_exposure_time.sidereal_time(siderealTime_type) 
    siderealTime_UTC_end = UTC_end_exposure_time.sidereal_time(siderealTime_type) 
    
    siderealTime_UTC_start_usr = UTC_Obs_usr.sidereal_time(siderealTime_type)
    siderealTime_UTC_mid_usr = UTC_mid_exposure_time_usr.sidereal_time(siderealTime_type) 
    
    # need to precess these
    object_observed_camera_start = SkyCoord(ra,dec,obstime=UTC_Obs,equinox='J2003.5',
                                            location=observatory_location)
    
    object_observed_camera_mid = SkyCoord(ra,dec,obstime=UTC_mid_exposure_time,equinox='J2003.5',
                                          location=observatory_location)
    
    # get the overall coordinates for the object
    object_Sky = SkyCoord(ra,dec,location=observatory_location,equinox='J2003.5')
    object_altaz_camera_mid = object_Sky.transform_to(AltAz(obstime=(UTC_mid_exposure_time),
                                                            location=observatory_location))
    alt = object_altaz_camera_mid.alt
    alt_str = alt.to_string(u.degree, alwayssign=True)
    
    # airmass calculations
    computed_airmass_mid = object_altaz_camera_mid.secz
    airmass_diff = computed_airmass_mid - airmass
    airmass_diff_usr = _airmass - airmass
    
    # compute the 'real' hourangle start and mid, based on the .fits file information
    hourangle_start = siderealTime_UTC_start - object_observed_camera_start.ra
    hourangle_mid = siderealTime_UTC_mid - object_observed_camera_mid.ra
    
    # convert the entered beginning hour angle to a Longitude object
    bha_usr = Longitude(_bha,unit=u.hourangle)
    # compute the difference between the computed start and entered hour angle
    hourangle_start_usr_diff = hourangle_start - bha_usr
    
    print('---------------------------------------------------')
    print('File:        ', _filename)
    print('Object:      ', obj_name)
    print("LZT (hdr): {} {}".format(LZT_obs.iso, dst_string))
    print("LZT (usr): {} {}".format(LZT_obs_usr.iso, dst_string))
    print('Exposure:  ', exposure_time)
    print('Exposure (usr)', exposure_time_usr)
    print('Exposure diff:', exposure_time_usr_diff)
    print('HA (calc) :', hourangle_start)
    print('HA (usr): ', bha_usr)
    print('HA diff: ', hourangle_start_usr_diff)
    print('Airmass mid (hdr): ', airmass)
    print('Airmass mid (usr): ', _airmass)
    print('Airmass mid (calc): ', computed_airmass_mid)
    print('Airmass diff (calc): ', airmass_diff)
    print('Airmass diff (usr): ', airmass_diff_usr)
    # print('---------------------------------------------------')

    # additional code to display the image and a stretched version side by side.
    # the following is needed to stretch the image and then display it. it has nothing to do
    # with astronomical calculations. It does show how straightforward it is to display the
    # image data associated with the .fits file however
    
    if Plot_images:
        norm = ImageNormalize(data, interval=PercentileInterval(.1))
        fig = plt.figure(figsize=(8,3),dpi=120)
        ax1 = fig.add_subplot(1, 2, 1)
        ax1.imshow(data, origin='lower')
        ax2 = fig.add_subplot(1,2,2)
        im2 = ax2.imshow(data, origin='lower',norm=norm)
        fig.colorbar(im2)
        plt.show()
    
    Verbose = False
    if Verbose:
        print('RA:                    ', ra)
        print('Dec:                   ', dec)
        print('HA mid:                ', hourangle_mid)
        print('Altitude (mid exp):    ', alt_str)
        print('Exposure:              ', exposure_time)
        print('Airmass:               ', airmass)
        print('Airmass mid,(computed): ',computed_airmass_mid)
        print('UTC start exp:          ', UTC_Obs.iso)
        print('UTC mid exp:            ', UTC_mid_exposure_time.iso)
        print('UTC end exp:            ', UTC_end_exposure_time.iso)
        print('LZT start exp:          ', LZT_obs.iso)
        print('LZT mid exp:            ', LZT_obs_mid.iso)
        print('LZT end exp:            ', LZT_obs_end.iso)
        print('LST start:              ', siderealTime_local_start)
        print('LST mid:                ', siderealTime_local_mid)
        print('LST end:                ', siderealTime_local_end)
        print('---------------------------------------------------')
    else:
        print('Goodbye. Clear skies!')
else:
    print('Nothing to do')
    
### end of code


******************* HeadCheck Input *******************
p --- Enter path
f --- Enter a filename to be processed 
l --- List files
h --- View Header 
t --- Enter date & time 
j --- Enter Julian Day 
b --- Enter Beginning Hour Angle 
s --- Enter Airmass 
x --- Enter Exposure (sec) 
v --- Review Input 
P --- Process input 
z --- Reset all 
q --- Quit
Enter a command, q to exit, P to exit and process: q
Nothing to do


In [ ]:
2003-07-28T05:56:02.33
L110_4.fits
04:00:00

In [ ]:
_utc_time_obs
Time(_utc_time_obs)

In [ ]:
time = '2003-07-28T05:56:02.33'
utc = Time(_utc_time_obs,format='fits',location=observatory_location)

In [ ]:
utc